# Importing Liabraries


In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge


# Loading Dataset 

In [67]:
# Load your dataset
df = pd.read_csv('Bengaluru_House_Data.csv')
print(df)
# Preprocess the dataset as needed (e.g., handle missing values, encode categorical variables, etc.)


       area  bedrooms   price
0    1056.0         2   39.07
1    2600.0         5  120.00
2    1440.0         2   62.00
3    1521.0         3   95.00
4    1200.0         2   51.00
..      ...       ...     ...
294  1425.0         2   70.00
295  1450.0         3  100.00
296  1847.0         3  105.00
297  1100.0         2   45.00
298   525.0         1   26.00

[299 rows x 3 columns]


In [68]:
# Assuming 'Price' is the target variable and the rest are features
X = df.drop('price', axis=1)
y = df['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Applying the Ridge Regression

In [69]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
ridge_predictions = ridge.predict(X_test)
ridge_mse = mean_squared_error(y_test, ridge_predictions)
ridge_r2 = r2_score(y_test, ridge_predictions)

print(f'Ridge MSE: {ridge_mse}')
print(f'Ridge R²: {ridge_r2}')


Ridge MSE: 9262.194038432413
Ridge R²: 0.4031962375976498


# First Order (SGD)

In [70]:
# Initialize and train the model
sgd = SGDRegressor(max_iter=10000, tol=1e-3)
sgd.fit(X_train, y_train)

# Evaluate the model
sgd_score = sgd.score(X_test, y_test)
print(f'SGD Score: {sgd_score}')


SGD Score: -4.793943337942618e+26


# Advance Algorithm (Adam)

In [71]:
# Initialize and train the model
adam_model = MLPRegressor(solver='adam', max_iter=1000)
adam_model.fit(X_train, y_train)

# Evaluate the model
adam_score = adam_model.score(X_test, y_test)
print(f'Adam Optimizer Score: {adam_score}')


Adam Optimizer Score: 0.32684297870899504


In [72]:
'''Price=Intercept+(Slope Coefficient×Feature)'''

# Define the objective function
def objective_function(params):
    # Ensure params is a 1D array
    params = params.flatten()
    # Make predictions using the linear model
    predictions = X_train.dot(params)
    # Calculate the loss (mean squared error)
    loss = np.mean((y_train - predictions) ** 2)
    return loss

# Define a function to compute the gradient numerically
def numerical_gradient(params):
    epsilon = 1e-8
    params = params.flatten()  # Ensure params is 1D
    grad = np.zeros_like(params)
    for i in range(len(params)):
        params_plus = np.copy(params)
        params_plus[i] += epsilon
        grad[i] = (objective_function(params_plus) - objective_function(params)) / epsilon
    return grad

# Convert training data to numpy arrays if not already
X_train = np.array(X_train)
y_train = np.array(y_train)

# Initial parameter guess
initial_params = np.zeros(X_train.shape[1])

# Perform optimization
result = minimize(objective_function, initial_params, method='L-BFGS-B', jac=numerical_gradient)

# Check if optimization was successful
if result.success:
    optimized_params = result.x
    print(f'Optimization successful. Optimized parameters: {optimized_params}')
else:
    print(f'Optimization failed. Message: {result.message}')

# Apply the optimized parameters to the model and evaluate
# Here you would typically use the optimized parameters to make predictions and evaluate the model


Optimization successful. Optimized parameters: [0.06216347 3.54875051]


# Calculate Metrics

In [73]:
# Compute predictions for L-BFGS
l_bfgs_predictions = X_test.dot(optimized_params)
l_bfgs_mse = mean_squared_error(y_test, l_bfgs_predictions)
l_bfgs_r2 = r2_score(y_test, l_bfgs_predictions)

print(f'\nL-BFGS MSE: {l_bfgs_mse}')
print(f'L-BFGS R²: {l_bfgs_r2}')


# Predictions and evaluations for Adam
adam_predictions = adam_model.predict(X_test)
adam_mse = mean_squared_error(y_test, adam_predictions)
adam_r2 = r2_score(y_test, adam_predictions)

print(f'\nAdam Optimizer MSE: {adam_mse}')
print(f'Adam Optimizer R²: {adam_r2}')


# Predictions and evaluations for SGD
sgd_predictions = sgd.predict(X_test)
sgd_mse = mean_squared_error(y_test, sgd_predictions)
sgd_r2 = r2_score(y_test, sgd_predictions)

print(f'\nSGD MSE: {sgd_mse}')
print(f'SGD R²: {sgd_r2}')



L-BFGS MSE: 10063.43790543349
L-BFGS R²: 0.3515685830220844

Adam Optimizer MSE: 10447.170983696056
Adam Optimizer R²: 0.32684297870899504

SGD MSE: 7.440039122162888e+30
SGD R²: -4.793943337942618e+26
